In [1]:
from xhtml2pdf import pisa
from xhtml2pdf.config.httpconfig import httpConfig

In [2]:
# State

In [3]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
start_all = time.time()

In [6]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
tester_objects = [povm_x, povm_y, povm_z]

In [7]:
true_object = get_z0_1q(c_sys)
num_data = [100, 1000]
n_rep = 100

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

qtomography_list = [
    StandardQst(tester_objects, on_para_eq_constraint=True),
    StandardQst(tester_objects, on_para_eq_constraint=False),
    StandardQst(tester_objects, on_para_eq_constraint=True),
    StandardQst(tester_objects, on_para_eq_constraint=False),
]
para_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

estimation_results_list = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")

  6%|▌         | 6/100 [00:00<00:01, 53.28it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LinearEstimator


  6%|▌         | 6/100 [00:00<00:01, 50.63it/s]

elapsed_time:0.021422032515207926[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LinearEstimator


  0%|          | 0/100 [00:00<?, ?it/s]/Users/tomoko/project/rcast/workspace/quara/quara/objects/state.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  np.array(vec_new, dtype=np.float64),
  2%|▏         | 2/100 [00:00<00:05, 18.69it/s]

elapsed_time:0.025610947608947755[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator


  2%|▏         | 2/100 [00:00<00:05, 17.23it/s]

elapsed_time:0.08575025002161661[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator


100%|██████████| 100/100 [00:05<00:00, 18.88it/s]

elapsed_time:0.08841514587402344[min]



In [15]:
report.export_report("sample.pdf",
    estimation_results_list,
    case_name_list,
    qtomography_list,
    para_list,
    estimator_list,
    true_object,
    tester_objects,
    num_data,
    n_rep,
)

100%|██████████| 100/100 [00:00<00:00, 5112.51it/s]
/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars

/Users/tomoko/project/rcast/workspace/quara/quara/objects/state.py:283: ComplexWarning:

Casting complex values to real discards the imaginary part



Completed to export pdf. (sample.pdf)


In [ ]:
all_elapsed_time = time.time() - start_all
print("all_elapsed_time:{0}".format(all_elapsed_time / 60) + "[min]")